<a href="https://colab.research.google.com/github/yassine8enaddi/NLP_TP1/blob/main/TP1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 98% 166M/169M [00:06<00:00, 30.4MB/s]
100% 169M/169M [00:07<00:00, 25.1MB/s]


In [6]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [7]:
!pip install pandas

In [8]:
!pip install nltk

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import re
import nltk
import spacy
import string

In [11]:
pd.options.mode.chained_assignment = None

In [12]:
df=pd.read_csv('/content/sample.csv')

In [13]:
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0
...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0


In [ ]:
df["text"]

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text, Length: 93, dtype: object

In [14]:
df["text_lower_case"] = df["text"].str.lower()

In [15]:
PUNCT_TO_REMOVE = string.punctuation

In [16]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [17]:
df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))

In [ ]:
df["text_wo_punct"]

0     AppleSupport causing the reply to be disregard...
1     105835 Your business means a lot to us Please ...
2     76328 I really hope you all change but Im sure...
3     105836 LiveChat is online at the moment  https...
4     VirginTrains see attached error message Ive tr...
                            ...                        
88    105860 I wish Amazon had an option of where I ...
89    They reschedule my shit for tomorrow httpstcoR...
90    105861 Hey Sara sorry to hear of the issues yo...
91    Tesco bit of both  finding the layout cumberso...
92    105861 If that doesnt help please DM your full...
Name: text_wo_punct, Length: 93, dtype: object

In [18]:
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [21]:
df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))

In [ ]:
df["text_wo_stop"].head()

0    AppleSupport causing reply disregarded tapped ...
1    105835 Your business means lot us Please DM na...
2    76328 I really hope change Im sure wont Becaus...
3    105836 LiveChat online moment httpstcoSY94VtU8...
4    VirginTrains see attached error message Ive tr...
Name: text_wo_stop, dtype: object

In [22]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [ ]:
cnt.most_common(10)

[('I', 34),
 ('us', 25),
 ('DM', 19),
 ('help', 17),
 ('httpstcoGDrqU22YpT', 12),
 ('AppleSupport', 11),
 ('Thanks', 11),
 ('phone', 9),
 ('Hi', 8),
 ('get', 8)]

In [23]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

In [24]:
df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df["text_wo_stopfreq"].head()

0    causing reply disregarded tapped notification ...
1    105835 Your business means lot Please name zip...
2    76328 really hope change Im sure wont Because ...
3    105836 LiveChat online moment httpstcoSY94VtU8...
4    VirginTrains see attached error message Ive tr...
Name: text_wo_stopfreq, dtype: object

In [25]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [26]:
df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["text_wo_stopfreqrare"].head()

0    causing reply disregarded tapped notification ...
1    105835 Your business means lot Please name zip...
2    76328 really hope change Im sure wont Because ...
3    105836 LiveChat online moment httpstcoSY94VtU8...
4    VirginTrains see attached error message Ive tr...
Name: text_wo_stopfreqrare, dtype: object

In [27]:
from nltk.stem.porter import PorterStemmer

In [28]:
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [29]:
df["text_stemmed"] = df["text"].apply(lambda text: stem_words(text))
df["text_stemmed"].head()

0    @applesupport caus the repli to be disregard a...
1    @105835 your busi mean a lot to us. pleas dm y...
2    @76328 i realli hope you all chang but i'm sur...
3    @105836 livechat is onlin at the moment - http...
4    @virgintrain see attach error message. i'v tri...
Name: text_stemmed, dtype: object

In [30]:
nlp = spacy.load("en_core_web_sm")

In [31]:
def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [32]:
df["lemmatized_text"] = df["text"].apply(lambda text: lemmatize_words(text))
df["lemmatized_text"].head()

0    @applesupport cause the reply to be disregard ...
1    @105835 your business mean a lot to we . pleas...
2    @76328 I really hope you all change but I be s...
3    @105836 LiveChat be online at the moment - htt...
4    @virgintrain see attach error message . I have...
Name: lemmatized_text, dtype: object

In [33]:
def remove_emoji(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    clean_text = re.sub(emoji_pattern, '', text)
    return clean_text

In [34]:
df["text_wo_emojis"] = df["text"].apply(remove_emoji)
df["text_wo_emojis"]

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text_wo_emojis, Length: 93, dtype: object

In [35]:
def remove_urls(text):
    # This pattern matches most URLs
    url_pattern = r'https?://\S+|www\.\S+'
    clean_text = re.sub(url_pattern, '', text)
    return clean_text

In [36]:
df["text_wo_urls"] = df ["text"].apply(remove_urls)
df["text_wo_urls"]

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment -  or...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89                They reschedule my shit for tomorrow 
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text_wo_urls, Length: 93, dtype: object

In [37]:
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text

In [38]:
df["text_wo_html_tags"] = df["text"].apply(remove_html_tags)
df["text_wo_html_tags"]

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text_wo_html_tags, Length: 93, dtype: object

In [39]:
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower_case,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_wo_stopfreqrare,text_stemmed,lemmatized_text,text_wo_emojis,text_wo_urls,text_wo_html_tags
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,AppleSupport causing the reply to be disregard...,AppleSupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...,@applesupport caus the repli to be disregard a...,@applesupport cause the reply to be disregard ...,@AppleSupport causing the reply to be disregar...,@AppleSupport causing the reply to be disregar...,@AppleSupport causing the reply to be disregar...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,105835 Your business means a lot to us Please ...,105835 Your business means lot us Please DM na...,105835 Your business means lot Please name zip...,105835 Your business means lot Please name zip...,@105835 your busi mean a lot to us. pleas dm y...,@105835 your business mean a lot to we . pleas...,@105835 Your business means a lot to us. Pleas...,@105835 Your business means a lot to us. Pleas...,@105835 Your business means a lot to us. Pleas...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,76328 I really hope you all change but Im sure...,76328 I really hope change Im sure wont Becaus...,76328 really hope change Im sure wont Because ...,76328 really hope change Im sure wont Because ...,@76328 i realli hope you all chang but i'm sur...,@76328 I really hope you all change but I be s...,@76328 I really hope you all change but I'm su...,@76328 I really hope you all change but I'm su...,@76328 I really hope you all change but I'm su...
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,105836 LiveChat is online at the moment https...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...,@105836 livechat is onlin at the moment - http...,@105836 LiveChat be online at the moment - htt...,@105836 LiveChat is online at the moment - htt...,@105836 LiveChat is online at the moment - or...,@105836 LiveChat is online at the moment - htt...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,@virgintrain see attach error message. i'v tri...,@virgintrain see attach error message . I have...,@VirginTrains see attached error message. I've...,@VirginTrains see attached error message. I've...,@VirginTrains see attached error message. I've...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0,@105860 i wish amazon had an option of where i...,105860 I wish Amazon had an option of where I ...,105860 I wish Amazon option I get shipped ups ...,105860 wish Amazon option shipped ups store av...,105860 wish Amazon option shipped ups store av...,@105860 i wish amazon had an option of where i...,@105860 I wish Amazon have an option of where ...,@105860 I wish Amazon had an option of where I...,@105860 I wish Amazon had an option of where I...,@105860 I wish Amazon had an

In [40]:
df.drop(["inbound", "created_at","response_tweet_id", "in_response_to_tweet_id"], axis=1, inplace=True)

In [42]:
df.drop(["tweet_id", "author_id"], axis=1, inplace=True)

In [43]:
df

,text,text_lower_case,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_wo_stopfreqrare,text_stemmed,lemmatized_text,text_wo_emojis,text_wo_urls,text_wo_html_tags
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,AppleSupport causing the reply to be disregard...,AppleSupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...,@applesupport caus the repli to be disregard a...,@applesupport cause the reply to be disregard ...,@AppleSupport causing the reply to be disregar...,@AppleSupport causing the reply to be disregar...,@AppleSupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 Your business means a lot to us Please ...,105835 Your business means lot us Please DM na...,105835 Your business means lot Please name zip...,105835 Your business means lot Please name zip...,@105835 your busi mean a lot to us. pleas dm y...,@105835 your business mean a lot to we . pleas...,@105835 Your business means a lot to us. Pleas...,@105835 Your business means a lot to us. Pleas...,@105835 Your business means a lot to us. Pleas...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 I really hope you all change but Im sure...,76328 I really hope change Im sure wont Becaus...,76328 really hope change Im sure wont Because ...,76328 really hope change Im sure wont Because ...,@76328 i realli hope you all chang but i'm sur...,@76328 I really hope you all change but I be s...,@76328 I really hope you all change but I'm su...,@76328 I really hope you all change but I'm su...,@76328 I really hope you all change but I'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 LiveChat is online at the moment https...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...,@105836 livechat is onlin at the moment - http...,@105836 LiveChat be online at the moment - htt...,@105836 LiveChat is online at the moment - htt...,@105836 LiveChat is online at the moment - or...,@105836 LiveChat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,@virgintrain see attach error message. i'v tri...,@virgintrain see attach error message . I have...,@VirginTrains see attached error message. I've...,@VirginTrains see attached error message. I've...,@VirginTrains see attached error message. I've...
...,...,...,...,...,...,...,...,...,...,...,...
88,@105860 I wish Amazon had an option of where I...,@105860 i wish amazon had an option of where i...,105860 I wish Amazon had an option of where I ...,105860 I wish Amazon option I get shipped ups ...,105860 wish Amazon option shipped ups store av...,105860 wish Amazon option shipped ups store av...,@105860 i wish amazon had an option of where i...,@105860 I wish Amazon have an option of where ...,@105860 I wish Amazon had an option of where I...,@105860 I wish Amazon had an option of where I...,@105860 I wish Amazon had an option of where I...
89,They reschedule my shit for tomorrow https://t...,they reschedule my shit for tomorrow https://t...,They reschedule my shit for tomorrow httpstcoR...,They reschedule shit tomorrow httpstcoRsvZcT982t,They reschedule shit tomorrow httpstcoRsvZcT982t,They reschedule shit tomorrow httpstcoRsvZcT982t,they reschedul my shit for tomorrow https://t....,they reschedule my shit for tomorrow https://t...,They reschedule my shit for tomorrow https://t...,They reschedule my shit for t

In [44]:
df["text-cleaned"] = df["text"].str.lower()\
.apply(lambda text: remove_punctuation(text))\
.apply(lambda text: remove_stopwords(text))\
.apply(lambda text: remove_freqwords(text))\
.apply(lambda text: remove_rarewords(text))\
.apply(lambda text: lemmatize_words(text))\
.apply(remove_emoji)\
.apply(remove_urls)\
.apply(remove_html_tags)

In [45]:
df.drop(["lemmatized_text", "text_wo_emojis", "text_wo_urls", "text_wo_html_tags", "text_lower_case", "text_wo_punct", "text_wo_stop", "text_wo_stopfreq", "text_wo_stopfreqrare", "text_stemmed"], axis=1, inplace=True)

In [46]:
df

,text,text-cleaned
0,@AppleSupport causing the reply to be disregar...,applesupport cause reply disregard tap notific...
1,@105835 Your business means a lot to us. Pleas...,105835 business mean lot please dm name zip co...
2,@76328 I really hope you all change but I'm su...,76328 really hope change I m sure will not do not
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,virgintrain see attach error message I ve try ...
...,...,...
88,@105860 I wish Amazon had an option of where I...,105860 wish amazon option ship up store avoid ...
89,They reschedule my shit for tomorrow https://t...,reschedule shit tomorrow httpstcorsvzct982 t
90,"@105861 Hey Sara, sorry to hear of the issues ...",105861 hey sara sorry hear issue ask lay speed...
91,@Tesco bit of both - finding the layout cumber...,tesco bit find layout cumbersome remove item f...
